In [1]:
import binarize
import os
import pandas as pd
import re
import random

In [2]:
datapath = os.path.join(
    os.pardir,
    "train_test_data",
    "training1"
)

In [9]:
data = pd.read_csv(
    ".".join([
        datapath,
        "csv"
    ])
).dropna(
    axis=(
        0,
        1
    ),
    how="all"
)
data.drop(
    [
        col for col in data.columns if (
            "text" in col
        ) or (
            "Reason" in col
        ) or (
            col in [
                "EID",
                "Anx",
                "ASD",
                "train",
                "Dx2"
            ]
        )
    ],
    axis=1,
    inplace=True
)
data.Dx = [
    list(
        pd.Series(
            item
        ).apply(
            lambda x: re.sub(
                "n/a|N/A|n a|N A",
                "No Diagnosis Given",
                re.sub(
                    "  ",
                    " ",
                    re.sub(
                        "[\/\-\,\(\)/]",
                        " ",
                        x.title(
                        )
                    )
                )
            ).strip()
        )
    ) for item in [
        eval(d) for d in data.Dx
    ]
]
dxes = list(
    {
        re.sub(
            "  ",
            " ",
            re.sub(
                "[\/\-\,\(\)/]",
                " ",
                item.title(
                )
            )
        ).strip() for d in data[
            "Dx"
        ] for item in d if item.lower() != "n/a"
    }
)
random.shuffle(dxes)
data.Dx = data.Dx.apply(
    lambda m: [
        int(
            x == d
        ) for d in dxes for x in m
    ]
)

In [4]:
outpath = os.path.join(
    "Paweł",
    os.path.basename(
        datapath
    )
)
if not os.path.exists(
    os.path.dirname(
        outpath
    )
):
    os.makedirs(
        os.path.dirname(
            outpath
        )
    )
    
binarize.shuffle_dataframe_remove_labels(data).to_csv(
    ".".join([
        outpath,
        "csv"
    ]),
    header=False,
    index=False
)

binarize.shuffle_dataframe_remove_labels(
    binarize.binarize(data)
).to_csv(
    ".".join([
        "_".join([
            outpath,
            "binarized"
        ]),
        "csv"
    ]),
    header=False,
    index=False
)